### Regresión Logistica
Salazar Vega Rodrigo

In [25]:
# from nltk.Wrd import tokenizer
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from random import shuffle
from math import exp
import re

In [41]:
def read_corpus(path = str()):
    """
    path : ubicación del archivo a leer
    retorna una lista de textos
    """    
    with open('./corpus/SMS_Spam_Corpus_big.txt', 'r') as f:
        text = f.read()
    
    return text

def separar_texto(text = str()):
    """
    text = una cadena de texto plano que recien fue leida por python
    Separa el texto por renglon y regresa dos vectores, de la siguiente forma, primero el valor, despues la etiqueta
    ejemplo: return valor, etiquetas
    """
    # Creamos lista separando las por salto de linea
    lista_tex = text.split('\n')

    shuffle(lista_tex)

    # Creamos vectores a devolver
    valor = []
    etiquetas = []

    # Ciclo para dividir en etiqueta y valor
    for lista in lista_tex:
        valor_separado = lista.split(sep=',')

        # Obtenemos el tamaño de separar cada linea por comas, se le resta 1 por que python inicia las listas desde 0
        n = len(valor_separado)
        n = n-1
        
        # Si obtenemos 2 elementos en n, lo guardamos de esa manera sin usar slice ([:n-1])
        if n <= 1:
            texto = valor_separado[n-1]
            texto = ''.join(texto)
            eti = valor_separado[n]
        else:
            texto = valor_separado[:n-1]
            texto = ''.join(texto)
            eti = valor_separado[n]

        # Agregamos los valores obtenidos en el vector correspondiente
        valor.append(texto)
        etiquetas.append(eti)

    return valor, etiquetas

def cambio_a_numerico (vector = list()):
    """
    Función que requiere de un vector de etiquetas, para darles el valor de uno o cero.
    Por defecto: 1 si es spam y 0 si es ham.
    Retorna un vector del mismo tamaño que el vector de ingreso, pero con sus respectivos cambios.

    vector : lista de etiquetas, osea de puros valores string
    """
    resultado = []
    for valor in vector:
        if valor == 'spam':
            resultado.append(1)
        else:
            resultado.append(0)
    return np.array(resultado)


def tokenize(lines):
    new_lines = list()
    for line in valor:
        new_line = line.lower()
        new_lines.append(word_tokenize(new_line))
    return new_lines

def clean_text(lines):
    new_lines = list()
    for line in lines:
        new_line = list()
        for word in line:
            token = list()
            for c in word:
                #[a-záéíóúñü+$]
                if re.match(r'^[a-záéíóúñü+$]', c):
                    token.append(c)
            token = ''.join(token)
            if token != '':
                new_line.append(token)
        new_lines.append(new_line)
    return new_lines
    
def get_vocabulary(X):
    words = list()
    for line in X:
        for word in line:
            words.append(word)
    vocabulary = list(sorted(set(words)))
    return vocabulary


In [34]:
def lemmatize_X(X):
    lemmas_X = list()
    wnl = nltk.WordNetLemmatizer()
    for line in X:
        lemmas_line = list()
        for token in line:
            lemmatize_token = wnl.lemmatize(token)
            lemmas_line.append(lemmatize_token)
        lemmas_X.append(lemmas_line)
    return lemmas_X

def get_Corpus(X, vocabulary):
    matrix_X = list()
    for line in X:
        xi = list()
        xi.append(1)
        m = len(line)
        for word in vocabulary:
            if m != 0:
                xi.append(line.count(word) / m) 
            else:
                xi.append(m)
        xi = np.array(xi)
        matrix_X.append(xi)
    matrix_X =  np.array(matrix_X)
    return matrix_X.T

In [53]:
def initialize_w(n):
    w = np.zeros(n)
    return w

def get_z(Matrix, w):
    z = np.dot(w.T, Matrix)
    return z

def prediction(z):
    new_y = np.exp(z) / (1 + np.exp(z))
    return np.array(new_y)

def j(y, prediction):
    sum1 = np.sum(y * np.log(prediction))
    sum2 = np.sum((1 - y) * np.log(1 - prediction))
    result = - (sum1 + sum2) / len(y)
    return result

def j_partial(X, y, prediction):
    result = list()
    for x in X:
        result = (1 / len(y)) * np.dot((prediction - y), x)
    return np.array(result)

def get_new_w(w, jpartial, alpha = 0.0005):
    new_w =  w - (alpha * jpartial)
    return new_w

def train(X, y, w, iterations = 1000):
    for i in range(iterations):
        z = get_z(X, w)
        new_y = prediction(z)
        error = j(y, new_y)
        print(f'Error en la iteración{i+1}: {error}')
        new_w = j_partial(X, y, new_y)
        w = get_new_w(w, new_w)
    return w

In [62]:
def comprobacion(y_pred, y_test):
    """
    Obtención del conteo de los verdaderos positivos, falsos negativos, falsos positivos y falsos verdaderos
    Retora : TP, TN, FP, TN
    En ese orden
    TP : True Positive
    TN : True Negative
    FP : False Positive
    TN : True Negativa
    """

    n1 = len(y_pred)
    n2 = len(y_test)
    TP = 0
    FN = 0
    FP = 0
    TN = 0

    if n1 == n2:
        for i in n1:
            if y_pred[i] == 1 and y_test[i] == 1:
                TP += 1
                p += 1
            if y_pred[i] == 0 and y_test[i] == 0:
                TN += 1
            if y_pred[i] == 1 and y_test[i] == 0:
                FP += 1
            if y_pred[i] == 0 and y_test[i] == 1:
                FN += 1
    else:
        print('Error en los datos pasados')

    return TP, TN, FP, TN

In [64]:
def get_Accuracy(tp, tn, fp, fn):
    return (tp + tn) / (tp + tn + fp + fn)

def get_precition (tp, fp):
    return (tp / (tp + fp))

def get_recall (tp, fn):
    return (tp / (tp + fn))

def f1_measure (precition, recall):
    return (2 * ((precision * recall) / (precision + recall)))

In [63]:
def evaluation_result(spam_accuracy, spam_precision, spam_recall, spam_fmeasure, ham_accuracy, ham_precision, ham_recall, ham_fmeasure):
    output = ""

    output += "################################################################################## \n"
    output += "#                           *** Evaluation Results ***                           # \n"
    output += "#                                                                                # \n"
    output += "#                  Accuracy |     Precission    | Recall |     F1-measure        # \n"
    output += "# ==========================|===================|========|====================== # \n"
    output += f"#  Spam Class :    {spam_accuracy}   |{spam_precision} | {spam_recall}   | {spam_fmeasure}    # \n"
    output += "# --------------------------|-------------------|--------|---------------------- # \n"
    output += f"#  Ham  Class :    {ham_accuracy}   |{ham_precision} | {ham_recall}  | {ham_fmeasure}    # \n"
    output += "#                           |                   |        |                       # \n"
    output += "################################################################################## \n"

    return output

In [60]:
def spam_confusionMatrix(tp, tn, fp, fn):
    output = ""

    output += "             ########################################################### \n"
    output += "             #          *** Confusion Matrix (Spam Class) ***          # \n"
    output += "             #                                                         # \n"
    output += "             #                |    Spam     |     Ham     |            # \n"
    output += "             #      ==========|=============|=============|======      # \n"
    output += f"             #        Spam    |  TP = {tp}   |  FN = {fn}    |            # \n"
    output += "             #      ==========|=============|=============|======      # \n"
    output += f"             #        Ham     |  FP = {fp}     |  TN = {tn}   |            # \n"
    output += "             #                |             |             |            # \n"
    output += "             ########################################################### \n"

    return output

In [6]:
path = './corpus/SMS_Spam_Corpus_big.txt'

text = read_corpus(path)

In [42]:
valor, etiqueta = separar_texto(text)

In [43]:
et_num = cambio_a_numerico(etiqueta)

In [44]:
valor_token = tokenize(valor)
# valor_token

In [45]:
clean_lines = clean_text(valor_token)
# clean_lines

In [46]:
X_lemma = lemmatize_X(clean_lines)
# X_lemma

In [47]:
vocabulary = get_vocabulary(X_lemma)
# vocabulary

In [48]:
X = get_Corpus(X_lemma, vocabulary)


In [49]:
n = len(vocabulary)
w = initialize_w(n + 1)

In [52]:
w = train(X, et_num, w)

Error en la iteración0: 0.6915307912862605
Error en la iteración1: 0.6915291824280166
Error en la iteración2: 0.6915275735847829
Error en la iteración3: 0.6915259647565589
Error en la iteración4: 0.691524355943345
Error en la iteración5: 0.6915227471451407
Error en la iteración6: 0.6915211383619463
Error en la iteración7: 0.6915195295937611
Error en la iteración8: 0.6915179208405853
Error en la iteración9: 0.6915163121024187
Error en la iteración10: 0.6915147033792617
Error en la iteración11: 0.6915130946711134
Error en la iteración12: 0.6915114859779744
Error en la iteración13: 0.6915098772998438
Error en la iteración14: 0.6915082686367223
Error en la iteración15: 0.6915066599886092
Error en la iteración16: 0.6915050513555048
Error en la iteración17: 0.6915034427374086
Error en la iteración18: 0.6915018341343209
Error en la iteración19: 0.6915002255462412
Error en la iteración20: 0.6914986169731698
Error en la iteración21: 0.6914970084151063
Error en la iteración22: 0.6914953998720507

In [54]:
z = get_z(X, w)
z

array([-0.01260765, -0.01260765, -0.01260765, ..., -0.01260765,
       -0.01260765, -0.01260765])

In [58]:
new_y = prediction(z)
for i in range(len(et_num)):
    # if new_y[i] >= 0.4:
    print(et_num[i], new_y[i])

0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
1 0.4968481285114713
1 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
1 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
1 0.4968481285114713
1 0.4968481285114713
0 0.4968481285114713
1 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
1 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
1 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
1 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.4968481285114713
0 0.496848128